## Python 中的 GIL
- gil = global interpreter lock (cpython)
- python 中的一个线程对应于 c 语言中的一个线程
- gil 同一时刻，只有一个线程，在某一个cpu 上执行字节码
    - 无法将多个线程映射到多个 cpu 上执行

- 并发问题
    - 实际上，total += 1 操作并不是原子操作。它可以被分解为：
        - 读取 total 的当前值
        - 将当前值加 1
        - 将结果写回 total

    - 在多线程的环境中，如果 thread1 和 thread2 同时执行 total += 1 和 total -= 1，就会发生竞争条件。比如：
        - thread1 读取到 total 的值（假设是 0），然后线程切换到 thread2
        - thread2 读取到 total 的值（仍然是 0），然后 total 被减少 1，结果是 total 变为 -1
        - 然后切换回 thread1，它将读取到的值加 1，结果写回 total，最终 total 变成 1，而不是 0。

In [ ]:
import threading

total = 0

def add():
    global total
    for i in range(1000_000):
        total += 1

def desc():
    global total
    for i in range(1000_000):
        total -= 1

# scheduler 基于代码行数+时间片进行调度
thread1 = threading.Thread(target=add)
thread2 = threading.Thread(target=desc)
thread1.start()
thread2.start()

thread1.join()
thread2.join()
print("total = ", total)

total =  -98809
